In [ ]:
# Simple standalone function to create Jira incidents
def create_jira_incident(incident_details, config=None):
    """
    Create a Jira incident using ReActAgent with automatic tool execution.
    This is a standalone function that creates its own agent and session.
    
    Args:
        incident_details (dict): Dictionary containing incident details with keys:
            - project_key: Jira project key (e.g., 'KAN')
            - summary: Issue summary/title
            - description: Detailed description of the incident
            - issue_type: Type of issue (e.g., 'Incident', 'Bug', 'Task')
        config (dict, optional): Configuration overrides with keys:
            - base_url: LlamaStack server URL (default: 'http://localhost:8321')
            - model_id: Model ID (default: 'r1-qwen-14b-w4a16')
            - temperature: Temperature for sampling (default: 0.0)
            - max_tokens: Maximum tokens (default: 4096)
            - max_infer_iters: Max inference iterations (default: 10)
    
    Returns:
        dict: Response from the agent with execution details
    """
    
    # Import required modules
    import os
    from dotenv import load_dotenv
    from llama_stack_client import LlamaStackClient
    from llama_stack_client.lib.agents.react.agent import ReActAgent
    from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
    
    # Load configuration
    load_dotenv('config.env')
    
    # Set default configuration
    default_config = {
        'base_url': os.environ.get('LLAMA_STACK_URL', 'http://localhost:8321'),
        'model_id': os.environ.get('LLM_MODEL_ID', 'r1-qwen-14b-w4a16'),
        'temperature': float(os.environ.get("TEMPERATURE", 0.0)),
        'max_tokens': int(os.environ.get("MAX_TOKENS", 4096)),
        'max_infer_iters': int(os.environ.get("MAX_INFER_ITERATIONS", 10))
    }
    
    # Override with provided config
    if config:
        default_config.update(config)
    
    # Validate required fields
    required_fields = ['project_key', 'summary', 'description', 'issue_type']
    for field in required_fields:
        if field not in incident_details:
            raise ValueError(f"Missing required field: {field}")
    
    print("🚀 Initializing ReActAgent for Jira incident creation...")
    print(f"🔗 Connecting to: {default_config['base_url']}")
    print(f"🤖 Using model: {default_config['model_id']}")
    print(f"🔄 Max inference iterations: {default_config['max_infer_iters']}")
    
    try:
        # Create client
        client = LlamaStackClient(base_url=default_config['base_url'])
        print("✅ Client created")
        
        # Sampling parameters
        sampling_params = {
            "strategy": {"type": "greedy"},
            "max_tokens": default_config['max_tokens'],
            "temperature": default_config['temperature'],
        }
        
        # Create agent with AUTO tool execution
        agent = ReActAgent(
            client=client,
            model=default_config['model_id'],
            tools=["mcp::atlassian"],  # Specify tool groups
            response_format={
                "type": "json_schema",
                "json_schema": ReActOutput.model_json_schema(),
            },
            sampling_params=sampling_params,
            max_infer_iters=default_config['max_infer_iters']  # This is key for auto execution!
        )
        print("✅ ReActAgent created with AUTO tool execution")
        
        # Create session
        session_id = agent.create_session("jira-incident-creation")
        print(f"✅ Session created: {session_id}")
        
    except Exception as e:
        print(f"❌ Failed to initialize agent: {e}")
        import traceback
        traceback.print_exc()
        return {
            "success": False,
            "error": f"Agent initialization failed: {str(e)}",
            "traceback": traceback.format_exc()
        }
    
    # Build the simple prompt for creating the incident
    prompt = f"Create a Jira {incident_details['issue_type']} in the {incident_details['project_key']} project. Summary: {incident_details['summary']}. Description: {incident_details['description']}."
    
    print(f"\n🔧 Creating Jira incident with details:")
    print(f"   Project: {incident_details['project_key']}")
    print(f"   Type: {incident_details['issue_type']}")
    print(f"   Summary: {incident_details['summary']}")
    print(f"   Description: {incident_details['description']}")
    print(f"\n📝 Generated prompt: {prompt}")
    print("\n" + "="*60)
    
    try:
        # Create turn with the incident details
        response = agent.create_turn(
            messages=[{"role": "user", "content": prompt}],
            session_id=session_id,
            stream=False  # Non-streaming for auto execution
        )
        
        print("\n🤖 ReActAgent response:")
        print(f"Response: {response.output_message.content}")
        
        # Display execution steps
        print(f"\n🔍 Execution steps: {len(response.steps)}")
        for i, step in enumerate(response.steps):
            print(f"Step {i+1}: {step.step_type}")
            if hasattr(step, 'api_model_response') and step.api_model_response:
                content = step.api_model_response.content
                if len(content) > 200:
                    content = content[:200] + "..."
                print(f"  Content: {content}")
        
        return {
            "success": True,
            "response": response,
            "steps_count": len(response.steps),
            "output": response.output_message.content,
            "session_id": session_id,
            "agent_config": default_config
        }
        
    except Exception as e:
        print(f"❌ Failed to create Jira incident: {e}")
        import traceback
        traceback.print_exc()
        return {
            "success": False,
            "error": str(e),
            "traceback": traceback.format_exc(),
            "session_id": session_id,
            "agent_config": default_config
        }

print("✅ Simple standalone function 'create_jira_incident' defined successfully!")
print("   Usage: create_jira_incident(incident_details, config=None)")
print("   - Only requires: project_key, summary, description, issue_type")
print("   - No need to create agent or session manually")
print("   - Automatically handles all initialization")


In [ ]:
# Test the create_jira_incident function
print("🧪 Testing the create_jira_incident function")
print("="*60)

# Test incident details
test_incident = {
    "project_key": "KAN",
    "summary": "Test Incident - Function Validation",
    "description": "This is a test incident to validate that the create_jira_incident function works correctly. Testing automatic Jira creation with ReActAgent.",
    "issue_type": "Incident"
}

print("📋 Test incident details:")
for key, value in test_incident.items():
    print(f"   {key}: {value}")

print("\n" + "="*60)
print("🚀 Calling create_jira_incident function...")
print("\n")

try:
    # Call the function
    result = create_jira_incident(test_incident)
    
    print("\n" + "="*60)
    print("📊 Function Test Results:")
    print(f"   Success: {result['success']}")
    
    if result['success']:
        print(f"   Steps executed: {result['steps_count']}")
        print(f"   Session ID: {result['session_id']}")
        print(f"   Output: {result['output']}")
        print("\n✅ Function test PASSED!")
    else:
        print(f"   Error: {result['error']}")
        print("\n❌ Function test FAILED!")
        
except Exception as e:
    print(f"\n❌ Function call failed: {e}")
    import traceback
    traceback.print_exc()
